In [13]:
import arcpy
import pandas as pd
import requests
import zipfile
import os

# Base URL for Minnesota GIS data
BASE_URL = "https://gisdata.mn.gov/api/3/action/"

FOLDER_PATH = "GIS5572_Lab2"
DESKTOP_PATH = os.path.join(os.path.expanduser("~"), "Desktop")
DATASET_DIRECTORY = os.path.join(DESKTOP_PATH, FOLDER_PATH)

def pullFromMNGIS(datasetName: str):
    SAVE_DIR = os.path.join(DESKTOP_PATH, FOLDER_PATH, datasetName)
    os.makedirs(SAVE_DIR, exist_ok=True)

    # Fetch dataset details
    response = requests.get(f"{BASE_URL}package_show", params={"id": datasetName})
    data = response.json()

    if data["success"]:
        resources = data["result"]["resources"]

        # Find the TIFF or ZIP resource
        tiff_url = None
        for resource in resources:
            if "tif" in resource["url"].lower() or "zip" in resource["url"].lower():
                tiff_url = resource["url"]
                file_name = os.path.join(SAVE_DIR, os.path.basename(tiff_url))
                print(f"Downloading: {tiff_url}")

                # Download the file
                with requests.get(tiff_url, stream=True) as r:
                    r.raise_for_status()
                    with open(file_name, "wb") as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)
                print(f"Downloaded: {file_name}")

                # Unzip if it's a ZIP file
                if file_name.endswith(".zip"):
                    with zipfile.ZipFile(file_name, "r") as zip_ref:
                        zip_ref.extractall(SAVE_DIR)
                    print(f"Extracted: {SAVE_DIR}")

            else:
                print("Failed to fetch dataset information.")

def pullFromNOAA(datasetName:str, params: dict):
    SAVE_DIR = os.path.join(DESKTOP_PATH, FOLDER_PATH, datasetName)
    os.makedirs(SAVE_DIR, exist_ok=True)
    
    # NOAA API Key
    noaa_key = "jmDCMdCsdIhjIADxAZxYSxhztAALtLII"

    # API URL and parameters
    url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
    headers = {"token": noaa_key}

    # Make the request
    response = requests.get(url, headers=headers, params=params)

    # Check response
    if response.status_code == 200:
        data = response.json().get('results', [])
        if data:
            df = pd.json_normalize(data)
            df.to_csv(os.path.join(SAVE_DIR,"monthly_normals.csv"), index=False)
            print("Data saved as monthly_normals.csv at", SAVE_DIR)
        else:
            print("No data found for", SAVE_DIR)
    else:
        print(f"Error: {response.status_code}, {response.text}")

pullFromMNGIS("biota-bmsb")
pullFromMNGIS("biota-landcover-nlcd-mn-2019")
pullFromNOAA("NOAA_MPLSNORMALS", {
        "datasetid": "NORMAL_MLY",
        "stationid": "GHCND:USW00014922",  # MSP station ID
        "startdate": "2010-01-01",         # Normal period start date
        "enddate": "2019-12-31",           # Normal period end date
        "limit": 1000,                     # Increase if necessary
        "units": "metric"                  # Optional: set to 'standard' for Fahrenheit, inches, etc.
    })
pullFromNOAA("NOAA_MNNORMALS", {
    "datasetid": "NORMAL_MLY",
    "locationid": "FIPS:27",  # Minnesota FIPS code
    "startdate": "2010-01-01",
    "enddate": "2019-12-31"
})

pullFromMNGIS("biota-bmsb")

# Path to your GeoPackage DATASET
BMSB_DATASET_PATH = os.path.join(DATASET_DIRECTORY, "biota-bmsb")
gpkg_path = os.path.join(BMSB_DATASET_PATH, "BMSBSurveyDataTable.dbf")

BMSB_OUTPUT_PATH = os.path.join(DATASET_DIRECTORY, "OUTPUT", "bmsb_data.csv")

aprx = arcpy.mp.ArcGISProject("CURRENT")
map_obj = aprx.listMaps()[0]  # Assuming you want to add to the first map

map_obj.addDataFromPath(gpkg_path)

# CONVERT TO XY POINTS
arcpy.management.XYTableToPoint(
    in_table="BMSBSurveyDataTable",
    out_feature_class=r"BMSBSurveyData_XY_Points",
    x_field="Longitude",
    y_field="Latitude",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

# ADDS CTU DATA TO XY POINTS
arcpy.analysis.SpatialJoin(
    target_features="BMSBSurveyData_XY_Points",
    join_features="ctus_with_id",
    out_feature_class=r"BMSBSurveyData_SpatialJoin",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='SiteName "SiteName" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,SiteName,0,254;SiteType "SiteType" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,SiteType,0,254;City "City" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,City,0,254;County "County" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,County,0,254;Latitude "Latitude" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Latitude,-1,-1;Longitude "Longitude" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Longitude,-1,-1;SurveyName "SurveyName" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,SurveyName,0,254;Scientific "Scientific" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Scientific,0,254;CommonName "CommonName" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,CommonName,0,254;Surveyor "Surveyor" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Surveyor,0,254;Year "Year" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Year,-1,-1;TrapID "TrapID" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,TrapID,0,254;TrapType "TrapType" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,TrapType,0,254;Lure "Lure" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Lure,0,254;CheckDate "CheckDate" true true false 8 Date 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,CheckDate,-1,-1;Adults "Adults" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Adults,-1,-1;Nymphs "Nymphs" true true false 8 Double 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Nymphs,-1,-1;Notes "Notes" true true false 254 Text 0 0,First,#,BMSBSurveyDataTable_XYTableToPoint1,Notes,0,254;GNIS_FEATU "GNIS_FEATU" true true false 10 Long 0 10,First,#,ctus_with_id,GNIS_FEATU,-1,-1;FEATURE_NA "FEATURE_NA" true true false 254 Text 0 0,First,#,ctus_with_id,FEATURE_NA,0,254;CTU_CLASS "CTU_CLASS" true true false 25 Text 0 0,First,#,ctus_with_id,CTU_CLASS,0,25;COUNTY_GNI "COUNTY_GNI" true true false 10 Long 0 10,First,#,ctus_with_id,COUNTY_GNI,-1,-1;COUNTY_COD "COUNTY_COD" true true false 2 Text 0 0,First,#,ctus_with_id,COUNTY_COD,0,2;COUNTY_NAM "COUNTY_NAM" true true false 100 Text 0 0,First,#,ctus_with_id,COUNTY_NAM,0,100;POPULATION "POPULATION" true true false 10 Long 0 10,First,#,ctus_with_id,POPULATION,-1,-1;SHAPE_Leng "SHAPE_Leng" true true false 19 Double 0 0,First,#,ctus_with_id,SHAPE_Leng,-1,-1;SHAPE_Area "SHAPE_Area" true true false 19 Double 0 0,First,#,ctus_with_id,SHAPE_Area,-1,-1;ORIG_FID "ORIG_FID" true true false 10 Long 0 10,First,#,ctus_with_id,ORIG_FID,-1,-1;UNIQUE_ID "UNIQUE_ID" true true false 10 Long 0 10,First,#,ctus_with_id,UNIQUE_ID,-1,-1',
    match_option="WITHIN",
    search_radius=None,
    distance_field_name=""
)

# CLIPS OUT ANYTHING OUTSIDE OF MN BOUNDARIES
arcpy.analysis.PairwiseClip(
    in_features="BMSBSurveyData_SpatialJoin",
    clip_features=r"BOUNDARIES OF MINNESOTA",
    out_feature_class=r"BMSBSurveyData_PairwiseClip",
    cluster_tolerance=None
)

Downloading: https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_mda/biota_bmsb/shp_biota_bmsb.zip
Downloaded: C:\Users\18284\Desktop\GIS5572_Lab2\biota-bmsb\shp_biota_bmsb.zip
Extracted: C:\Users\18284\Desktop\GIS5572_Lab2\biota-bmsb
Downloading: https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_mda/biota_bmsb/fgdb_biota_bmsb.zip
Downloaded: C:\Users\18284\Desktop\GIS5572_Lab2\biota-bmsb\fgdb_biota_bmsb.zip
Extracted: C:\Users\18284\Desktop\GIS5572_Lab2\biota-bmsb
Downloading: https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_mda/biota_bmsb/gpkg_biota_bmsb.zip
Downloaded: C:\Users\18284\Desktop\GIS5572_Lab2\biota-bmsb\gpkg_biota_bmsb.zip
Extracted: C:\Users\18284\Desktop\GIS5572_Lab2\biota-bmsb
Failed to fetch dataset information.
Failed to fetch dataset information.
Failed to fetch dataset information.
Downloading: https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip
D

<Result 'C:\\Users\\18284\\Documents\\ArcGIS\\Projects\\MyProject2\\MyProject2.gdb\\BMSBSurveyData_PairwiseClip'>